In [17]:
import pyxnat
import os
import jsonlines
import json
import time
import pydicom
from typing import Dict, List

In [18]:
data_dir = '/images/blob/padchest/converted_2021-07-26/dicoms-fhir'
labels_file = '/images/blob/padchest/converted_2021-07-26/dicoms-fhir/labels.jsonl'

In [97]:
xnat_server = 'http://10.2.0.9'
user = 'admin'
password = 'admin'
project = 'padchest2'
xnat_interface = pyxnat.Interface(server=xnat_server, user=user, password=password)

In [87]:

def build_index() -> Dict[str, List]:
    label_dict = {}
    with jsonlines.open(labels_file) as reader:
        for obj in reader:
            
            image_url = obj['image_url']
            labels = obj['label']
            prefix = 'AmlDatastore://padchest/converted_2021-07-26/dicoms/'
            print(image_url[len(prefix):])
            image_id = image_url.split('/')[-2]
            label_dict[image_id] = labels
    return label_dict
index = build_index()


2.25.97841296295848262391905843189555177463/2.25.197429333288174753588417158830881632019/2.25.82126769351055526436383158006469583862.dcm
2.25.237225980595325597273708412121303747771/2.25.69600185875961495323782263943172095160/2.25.249174383596205994619998407754700005869.dcm
2.25.115984319239436468921171651977466682756/2.25.141347065401843142792669135503885869714/2.25.210151360244164762976887240731259767771.dcm
2.25.101833420846609716649715627267392185967/2.25.205592418462635608613582878534557287554/2.25.332164064417820129239802068545058771458.dcm
2.25.332519586475856940693594698450309979735/2.25.31472675347197840812563664364574808305/2.25.190708325456722401146844176909077221607.dcm
2.25.100454274432053212084790743710162533279/2.25.268225717012648043727076053692720577706/2.25.111560737383907677338472218743134259277.dcm
2.25.36226003092832338093534081112395856792/2.25.297031692297511539891871309571213583002/2.25.106606579614019590132459583155518343843.dcm
2.25.177998945719112668284728240

2.25.53804092071292508461811933234299950526/2.25.145706883565277643634402814054907938076/2.25.59375850841512985111896270718761234872.dcm
2.25.198106064639011018380823785548577903491/2.25.37815883173326975738402284365754071958/2.25.128549862687357681171378820796690426845.dcm
2.25.223402969780620964421271413743809598532/2.25.134933427912289436382991211209141735353/2.25.149277411228077632063283470208511189938.dcm
2.25.264922643922989382125747184142106764330/2.25.53926633116691925912322910257327053558/2.25.20164144626851601051242603495030765194.dcm
2.25.43020079393003193390265299158733610043/2.25.290136849487342346824318010687477254001/2.25.232438360992406697501627077824631368251.dcm
2.25.286926588457604685015452771502971252987/2.25.120656747825823130176665073563931144701/2.25.100164612677990516875179891426558800656.dcm
2.25.6570281965038505166505926235731501364/2.25.34633505174192313122363635355240410271/2.25.159423179978939473484920256721607978893.dcm
2.25.5468144689829363621876617749620

In [21]:
def read_dicom(file_path: str):
    with open(file_path, 'rb') as f:
        ds = pydicom.dcmread(file_path)
        return ds

In [22]:
def create_subject(name, project) -> str:
    uri = '/data/projects/%s/subjects/%s'%(project, name)
    response = xnat_interface.put(uri)
    subject_uid = response.content
    return str(subject_uid.decode())
    

In [40]:
def create_experiment(subject, experiment_id):
    print(subject_id)
    experiment = subject.experiment(experiment_id)
    options = {'xsiType': 'xnat:crSessionData'}
    experiment.create(**options)
    print('New experiment %s created' %experiment.id())
    return experiment

In [41]:
def create_scan(experiment, scan_id):
    options = {
        'xsiType': 'xnat:crScanData',
        'usability': 'usable'
    }
    scan = experiment.scan(scan_id)
    scan.create(**options)
    return scan

In [42]:
def clear_project():
    xnat_project = xnat_interface.select.project(project)
    print(list(xnat_project.subjects()))
    for subject in list(xnat_project.subjects()):
        subject.delete()
        print('deleted')

clear_project()

[<Subject Object> XNAT01_S00141 `Stroman228_Kurtis994` (project: scenario2) (Gender: U) 1 experiment http://10.2.0.9/data/projects/scenario2/subjects/XNAT01_S00141?format=html, <Subject Object> XNAT01_S00142 `Kunze215_Elden718` (project: scenario2) (Gender: U) 1 experiment http://10.2.0.9/data/projects/scenario2/subjects/XNAT01_S00142?format=html, <Subject Object> XNAT01_S00143 `Swift555_Donnie175` (project: scenario2) (Gender: U) 1 experiment http://10.2.0.9/data/projects/scenario2/subjects/XNAT01_S00143?format=html, <Subject Object> XNAT01_S00144 `Schowalter414_Russell422` (project: scenario2) (Gender: U) 1 experiment http://10.2.0.9/data/projects/scenario2/subjects/XNAT01_S00144?format=html, <Subject Object> XNAT01_S00145 `Okuneva707_Trina350` (project: scenario2) (Gender: U) 1 experiment http://10.2.0.9/data/projects/scenario2/subjects/XNAT01_S00145?format=html, <Subject Object> XNAT01_S00146 `Lubowitz58_Greg949` (project: scenario2) (Gender: U) 1 experiment http://10.2.0.9/data/pr

In [95]:
project = xnat_interface.select.project('padchest2')
subjects = project.subjects()
for subject in subjects:
    for experiment in subject.experiments():
        for scan in experiment.scans():
            id_ = scan.id().replace('_', '.')
            print(f'creating label for {id_}')
            label = index[id_]
            label_file = '/tmp/label.json'
            with open('/tmp/label.json', 'w') as f:
                json.dump(label, f)
        

            try:
                resource = scan.resource('LABEL')
                resource.create()
                f = resource.file('label.json')
                response = f.put(src=label_file, format='JSON', content='JSON', extract=False, overwrite=True)
            except Exception:
                print('Failed')
            else:
                print('Succeded')
        



creating label for 2.25.305307984233715575866436895013148178629
Failed
creating label for 2.25.214631097618828029598260715608822318864
Failed
creating label for 2.25.192231063686238359213327995682738012121
Failed
creating label for 2.25.244480674969191966880092685099992711023
Failed
creating label for 2.25.152769524351990736351165367861563254013
Failed
creating label for 2.25.172471882061777945660473329684218324745
Failed
creating label for 2.25.292401953150861310105895588150603752848
Failed
creating label for 2.25.300161708988940644333036515302329588926
Failed
creating label for 2.25.6328380530298980283066110703905898816
Succeded
creating label for 2.25.145638117649727110143439113733964907443
Failed
creating label for 2.25.31116149252745065360522636171105132684
Succeded
creating label for 2.25.9102763290507784543919350587188710063
Failed
creating label for 2.25.141347065401843142792669135503885869714
Succeded
creating label for 2.25.67570235204737764761268312084904600976
Succeded
crea

In [47]:
count = 0
experiments = {}

label_index = build_index()
for root, dirs, files in os.walk(data_dir):
    if files and files[0].endswith('.dcm'):
        dicom_file = files[0]
        file_path = root + '/'+ dicom_file
        if dicom_file not in label_index:
            continue
        count += 1
        dicom = read_dicom(file_path)
        subject_name = str(dicom['PatientName'].value).replace('^', '_')
        subject_id = None
        try:
            print(f'creating {subject_name}')
            subject_id = create_subject(subject_name, project)
        except Exception as e:
            print(f'Failed to create subject {e}')
            continue
        subject = xnat_interface.select.project(project).subject(subject_id)
        experiment = None
        try:
            experiment = create_experiment(subject, f'exp{count}')
            experiments[subject] = experiment
        except Exception as e:
            print(f'Failed to create experiment {e}')
            continue

        scan = create_scan(experiment, dicom_file.split(".dcm")[0].replace('.', '_'))
        print(scan)
        resource = scan.resource('DICOM')
        resource.create()

        f = resource.file(dicom_file)
        response = f.put(src=file_path, format='DICOM', content='RAW', extract=False, overwrite=True)
        file_uid = response
        label = label_index[dicom_file]
      
        label_file = '/tmp/label.json'
        with open('/tmp/label.json', 'w') as f:
            json.dump(label, f)
        
        resource = scan.resource('LABEL')
        resource.create()
        f = resource.file('label.json')
        response = f.put(src=label_file, format='JSON', content='JSON', extract=False, overwrite=True)
         
    if count == 10:
        break

creating Stroman228_Kurtis994
XNAT01_S00151
New experiment XNAT01_E00146 created
<Scan Object> 2_25_269000718940131394502394471219865557303 (``  frames)  http://10.2.0.9/data/projects/scenario2/subjects/XNAT01_S00151/experiments/exp1/scans/2_25_269000718940131394502394471219865557303?format=html
creating Kunze215_Elden718
XNAT01_S00152
New experiment XNAT01_E00147 created
<Scan Object> 2_25_3306247422047918493529936784705070920 (``  frames)  http://10.2.0.9/data/projects/scenario2/subjects/XNAT01_S00152/experiments/exp2/scans/2_25_3306247422047918493529936784705070920?format=html
creating Swift555_Donnie175
XNAT01_S00153
New experiment XNAT01_E00148 created
<Scan Object> 2_25_228574107663032879690392082227436105293 (``  frames)  http://10.2.0.9/data/projects/scenario2/subjects/XNAT01_S00153/experiments/exp3/scans/2_25_228574107663032879690392082227436105293?format=html
creating Schowalter414_Russell422
XNAT01_S00154
New experiment XNAT01_E00149 created
<Scan Object> 2_25_14174093705556

In [46]:
dicom_file.split(".dcm")[0].replace('.', '_')

'2_25_3012003400308846774347472988432950228'

In [102]:
label_dict

{'2.25.82126769351055526436383158006469583862.dcm': ['No_Finding'],
 '2.25.249174383596205994619998407754700005869.dcm': ['Opacity'],
 '2.25.210151360244164762976887240731259767771.dcm': ['Opacity'],
 '2.25.332164064417820129239802068545058771458.dcm': ['No_Finding'],
 '2.25.190708325456722401146844176909077221607.dcm': ['No_Finding'],
 '2.25.111560737383907677338472218743134259277.dcm': ['Opacity'],
 '2.25.106606579614019590132459583155518343843.dcm': ['Atelectasis',
  'Pleural_Effusion',
  'Pleural_Abnormalities'],
 '2.25.288856916057862677977653594428399214627.dcm': ['Atelectasis',
  'Pleural_Effusion',
  'Pleural_Abnormalities'],
 '2.25.212461397906913979117946429815718832885.dcm': ['No_Finding'],
 '2.25.21295911399383494291943739496412220897.dcm': ['Pleural_Effusion',
  'Pleural_Abnormalities',
  'Opacity'],
 '2.25.266104544833385492718727725618939911589.dcm': ['Pleural_Abnormalities',
  'Pleural_Effusion',
  'Opacity'],
 '2.25.255120326809463454205126847299028003227.dcm': ['Pleur